[Norvig's setup code](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb)

In [1]:
#  Python 3.x
import re
import numpy as np
import math
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    filename = './input{}.txt'.format(day)
    return open(filename)

def transpose(matrix): return zip(*matrix)

def first(iterable): return next(iter(iterable))

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

cat = ''.join

Ø   = frozenset() # Empty set
inf = float('inf')
BIG = 10 ** 999

def grep(pattern, lines):
    "Print lines that match pattern."
    for line in lines:
        if re.search(pattern, line):
            print(line)

def groupby(iterable, key=lambda it: it):
    "Return a dic whose keys are key(it) and whose values are all the elements of iterable with that key."
    dic = defaultdict(list)
    for it in iterable:
        dic[key(it)].append(it)
    return dic

def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c

# 2-D points implemented using (x, y) tuples
def X(point): return point[0]
def Y(point): return point[1]

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def euclidean_distance(p, q=(0, 0)): 
    "Euclidean (hypotenuse) distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def astar_search(start, h_func, moves_func):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(previous, s)
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost + h_func(s2), s2))
                path_cost[s2] = new_cost
                previous[s2] = s
    return dict(fail=True, front=len(frontier), prev=len(previous))
                
def Path(previous, s): 
    "Return a list of states that lead to state s, according to the previous dict."
    return ([] if (s is None) else Path(previous, previous[s]) + [s])

# [Day 1](https://adventofcode.com/2018/day/1)

For example, if the device displays frequency changes of +1, -2, +3, +1, then starting from a frequency of zero, the following changes would occur:

Current frequency  0, change of +1; resulting frequency  1.
Current frequency  1, change of -2; resulting frequency -1.
Current frequency -1, change of +3; resulting frequency  2.
Current frequency  2, change of +1; resulting frequency  3.
In this example, the resulting frequency is 3.

Here are other example situations:

+1, +1, +1 results in  3
+1, +1, -2 results in  0
-1, -2, -3 results in -6
Starting with a frequency of zero, what is the resulting frequency after all of the changes in frequency have been applied?


In [15]:
def day1_pt1(in_):
  return np.sum(list(map(int, in_)))

day1_pt1(Input(1))

590

## Part 2
Here are other examples:

* +1, -1 first reaches 0 twice.
* +3, +3, +4, -2, -4 first reaches 10 twice.
* -6, +3, +8, +5, -6 first reaches 5 twice.
* +7, +7, -2, -7, -4 first reaches 14 twice.

What is the first frequency your device reaches twice?

In [35]:
def day1_pt2(in_):
  xor_ = set()
  sum_ = 0
  for i in itertools.cycle(map(int, in_)):
    xor_.add(sum_)
    sum_ = sum_ + i
    if sum_ in xor_:
      return sum_

day1_pt2(list(Input(1)))

83445

# [Day 2](https://adventofcode.com/2018/day/2)

For example, if you see the following box IDs:

* abcdef contains no letters that appear exactly two or three times.
* bababc contains two a and three b, so it counts for both.
* abbcde contains two b, but no letter appears exactly three times.
* abcccd contains three c, but no letter appears exactly two times.
* aabcdd contains two a and two d, but it only counts once.
* abcdee contains two e.
* ababab contains three a and three b, but it only counts once.

Of these box IDs, four of them contain a letter which appears exactly twice, and three of them contain a letter which appears exactly three times. Multiplying these together produces a **checksum of 4 * 3 = 12.**

What is the checksum for your list of box IDs?

In [8]:
def day2_pt1(in_):
  def count23(s):
    counts = {v: k for k, v in Counter(s).items() if v in (2, 3)}
    return (
      counts.get(2, None) is not None, 
      counts.get(3, None) is not None)
  
  twos, threes = 0, 0
  for c2, c3 in map(count23, in_):
    twos += c2
    threes += c3
  return twos * threes

day2_pt1(Input(2))

6000

In [11]:
def diff_letters(s1, s2):
  diff = []
  for i, (c1, c2) in enumerate(zip(s1, s2)):
    if c1 != c2:
      diff.append(i)
    if len(diff) > 1:
      break
  return diff
  
def day2_pt2(in_):
  for i in range(len(in_)):
    for j in range(i + 1, len(in_)):
      diffs = diff_letters(in_[i], in_[j])
      if len(diffs) == 1:
        diff = diffs[0]
        return in_[i][:diff] + in_[i][diff+1:]

day2_pt2(list(Input(2)))        

'pbykrmjmizwhxlqnasfgtycdv\n'

# [Day 3](https://adventofcode.com/2018/day/3)


In [26]:
def parse_coordinates(line):
  # example: "#1 @ 850,301: 23x12"
  line = line.strip()
  
  at = line.index("@")
  colon = line.index(':')
  
  x, y = map(int, line[at+1:colon].split(','))
  w, h = map(int, line[colon+1:].split('x'))
  return int(line[1:at]), (x, y), (w, h)
  
def day3_pt1(in_):
  grid = np.zeros((1000, 1000), dtype=np.int32)
  for i, (x, y), (w, h) in map(parse_coordinates, in_):
    grid[x:x+w, y:y+h] += 1
  return np.count_nonzero(grid > 1)

day3_pt1(Input(3))

113576

In [36]:
def day3_pt2(in_):
  grid = np.zeros((1000, 1000), dtype=np.int32)
  non_overlap = set()
  for id_, (x, y), (w, h) in map(parse_coordinates, in_):
    uniques = np.unique(grid[x:x+w, y:y+h])
    if len(uniques) == 1 and uniques[0] == 0:
      non_overlap.add(id_)
    grid[x:x+w, y:y+h] = id_
    non_overlap.difference_update(uniques)
      
  return non_overlap

day3_pt2(Input(3))

{825}

# [Day 4](https://adventofcode.com/2018/day/4)

Using the format:
```
[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up
```

Timestamps are written using `year-month-day hour:minute` format. The guard falling asleep or waking up is always the one whose shift most recently started. Because all asleep/awake times are during the midnight hour (00:00 - 00:59), **only the minute portion (00 - 59) is relevant** for those events.

While this example listed the entries in chronological order, **your entries are in the order you found them. You'll need to organize them before they can be analyzed.**

Strategy 1: Find the guard that has the most minutes asleep. What minute does that guard spend asleep the most?

What is the ID of the guard you chose multiplied by the minute you chose? (In the above example, the answer would be 10 * 24 = 240.)

In [3]:
in4 = list(Input(4))

Note, the sorting could technically have been done without extraction, as date format is ISO8601.

In [4]:
Event = namedtuple('Event', 'year month day hour minute action guard')
SWITCH, SLEEP, WAKE = -1, -2, -3


def structure(line):
  regex = r'\[(\d+)\-(\d+)\-(\d+) (\d+):(\d+)\][^#]*#?(\d*).*'
  year, month, day, hour, minute, guard = map(
    lambda x: int(x) if x else None, 
    re.match(regex, line).groups())
  action = SWITCH if guard else SLEEP if 'sleep' in line else WAKE
  return Event(year, month, day, hour, minute, action, guard)


def day4_pt1(in_):
  guards = defaultdict(lambda: np.array([0] * 60))
  asleep_min = defaultdict(int)
  on_duty = None
  for event in sorted(map(structure, in_)):
    if event.action == SWITCH:
      on_duty = event.guard
    elif event.action == SLEEP:
      sleep_minute = event.minute
    else:
      guards[on_duty][sleep_minute:event.minute] += 1
      asleep_min[on_duty] += (event.minute - sleep_minute)
  
  longest_asleep_id, _ = max(asleep_min.items(), key=lambda x: x[1])
  return longest_asleep_id * np.argmax(guards[longest_asleep_id])

day4_pt1(in4)

94040

In [5]:
def day4_pt2(in_):
  guards = defaultdict(lambda: np.array([0] * 60))
  on_duty = None
  for event in sorted(map(structure, in_)):
    if event.action == SWITCH:
      on_duty = event.guard
    elif event.action == SLEEP:
      sleep_minute = event.minute
    else:
      guards[on_duty][sleep_minute:event.minute] += 1
  
  longest_asleep_id, _ = max(guards.items(), key=lambda x: max(x[1]))
  return longest_asleep_id * np.argmax(guards[longest_asleep_id])

day4_pt2(in4)

39940

# [Day5](https://adventofcode.com/2018/day/5)

### For example:
- In aA, a and A react, leaving nothing behind.
- In abBA, bB destroys itself, leaving aA. As above, this then destroys itself, leaving nothing.
- In abAB, no two adjacent units are of the same type, and so nothing happens.
- In aabAAB, even though aa and AA are of the same type, their polarities match, and so nothing happens.

Now, consider a larger example, dabAcCaCBAcCcaDA:

```
dabAcCaCBAcCcaDA  The first 'cC' is removed.
dabAaCBAcCcaDA    This creates 'Aa', which is removed.
dabCBAcCcaDA      Either 'cC' or 'Cc' are removed (the result is the same).
dabCBAcaDA        No further actions can be taken.
After all possible reactions, the resulting polymer contains 10 units.
```

How many units remain after fully reacting the polymer you scanned? 

In [3]:
in5 = Input(5).read().strip()

In [12]:
def day5_pt1(s):
  stack = ['']
  for c in s:
    same_letter = stack[-1].lower() == c.lower()
    opposite_polarities = stack[-1] != c
    if same_letter and opposite_polarities:
      stack.pop()
    else:
      stack.append(c)
      
  return len(stack) - 1

day5_pt1(in5)

11590

## Part 2

One of the unit types is causing problems; it's preventing the polymer from collapsing as much as it should. Your goal is to figure out which unit type is causing the most problems, remove all instances of it (regardless of polarity), fully react the remaining polymer, and measure its length.

For example, again using the polymer dabAcCaCBAcCcaDA from above:

- Removing all A/a units produces dbcCCBcCcD. Fully reacting this polymer produces dbCBcD, which has length 6.
- Removing all B/b units produces daAcCaCAcCcaDA. Fully reacting this polymer produces daCAcaDA, which has length 8.
- Removing all C/c units produces dabAaBAaDA. Fully reacting this polymer produces daDA, which has length 4.
- Removing all D/d units produces abAcCaCBAcCcaA. Fully reacting this polymer produces abCBAc, which has length 6.

In this example, removing all C/c units was best, producing the answer 4.

What is the length of the shortest polymer you can produce by removing all units of exactly one type and fully reacting the result?

In [34]:
def day5_pt2(s):
  all_chars = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
  replace_char = lambda c: s.replace(c, '').replace(c.lower(), '')
  lengths = map(day5_pt1, map(replace_char, all_chars))
  return min(zip(lengths, all_chars))

day5_pt2(in5)

(4504, 'N')